## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
path = os.path.join("..", "Weather_Database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(path)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,0,14.97,clear sky
1,1,Saskylakh,RU,71.9167,114.0833,49.78,71,82,7.90,broken clouds
2,2,Sol-Iletsk,RU,51.1631,54.9918,80.35,38,18,2.80,few clouds
3,3,Port Hardy,CA,50.6996,-127.4199,60.80,83,55,2.68,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,73.09,79,99,24.90,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[(
    city_data_df["Max Temp"] >= min_temp) 
    & (city_data_df["Max Temp"] <= max_temp)]

prefered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Sol-Iletsk,RU,51.1631,54.9918,80.35,38,18,2.80,few clouds
5,5,Kulhudhuffushi,MV,6.6221,73.0700,81.57,84,31,11.77,scattered clouds
7,7,Hithadhoo,MV,-0.6000,73.0833,82.44,72,77,5.88,broken clouds
17,17,Albany,US,42.6001,-73.9662,75.36,69,61,5.14,broken clouds
19,19,Ribeira Grande,PT,38.5167,-28.7000,75.60,78,40,3.44,scattered clouds


In [13]:
# 4. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
prefered_cities_df = prefered_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prefered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sol-Iletsk,RU,80.35,few clouds,51.1631,54.9918,
5,Kulhudhuffushi,MV,81.57,scattered clouds,6.6221,73.0700,
7,Hithadhoo,MV,82.44,broken clouds,-0.6000,73.0833,
17,Albany,US,75.36,broken clouds,42.6001,-73.9662,
19,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
20,Kantang,TH,79.25,overcast clouds,7.4054,99.5156,
22,Atuona,PF,77.43,light rain,-9.8000,-139.0333,
28,Victoria,HK,82.26,overcast clouds,22.2855,114.1577,
30,Canton,CN,78.87,overcast clouds,23.1167,113.2500,
32,Cuamba,MZ,76.30,clear sky,-14.8031,36.5372,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in prefered_cities_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sol-Iletsk,RU,80.35,few clouds,51.1631,54.9918,Raduzhnaya
5,Kulhudhuffushi,MV,81.57,scattered clouds,6.6221,73.0700,Haajy Guest House 1
7,Hithadhoo,MV,82.44,broken clouds,-0.6000,73.0833,Scoop Guest House
19,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
20,Kantang,TH,79.25,overcast clouds,7.4054,99.5156,Reng Ta Chai Boutique HomeStay
...,...,...,...,...,...,...,...
650,San Fernando,PH,78.53,light rain,15.0286,120.6898,Maharlika Hostel
652,Hazorasp,UZ,82.24,clear sky,41.3194,61.0742,Kemping wifi hotel
655,Beyla,GN,75.85,moderate rain,8.6833,-8.6333,DIAN SINKO
657,Puerto Colombia,CO,82.49,overcast clouds,10.9878,-74.9547,Hotel El Emigrante


In [25]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("WeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<h3 id="firstHeading" class="firstHeading">{Hotel Name}</h3>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))